## Question 3 - HW2

Here I am going to implement a siamese neural network and train it with vocalization data.

In [87]:
# import libraries

import arff
import numpy as np
from sklearn.preprocessing import normalize

In [581]:
# 'attributes', 'relation', 'description', and 'data'

def countLabels(data_list):
    crying_class = {}
    
    for data in data_list:
        for record in data:
            label = record[-1]
        
            crying_class.setdefault(label, 0)
            crying_class[label] += 1
        
    return crying_class

def getDataAndLabel(data):
    X = []
    y = []
    
    for record in data:
        X.append(record[1:-1])
        y.append(int(record[-1]))
    
    return X, y
    
# for subject_out in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']:
for subject_out in ['10']:
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    for subject_in in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']:
        file_name_tr = 'arff/train_loso/ComParE2018_Crying.ComParE.' + subject_in + '.tr.arff'
        file_name_te = 'arff/train_loso/ComParE2018_Crying.ComParE.' + subject_in + '.te.arff'
        
        if subject_out == subject_in:
            data1 = arff.load(open(file_name_tr, 'rb'))
            data2 = arff.load(open(file_name_te, 'rb'))

            data1, label1 = getDataAndLabel(data1['data'])
            data2, label2 = getDataAndLabel(data2['data'])

            data1 = normalize(data1, axis = 0, norm = 'max').tolist()
            data2 = normalize(data2, axis = 0, norm = 'max').tolist()

            X_test += data1 + data2
            y_test += label1 + label2
            
            print ("subject " + str(subject_in) + (" finished. (test set)"))
            
            continue
            
        data1 = arff.load(open(file_name_tr, 'rb'))
        data2 = arff.load(open(file_name_te, 'rb'))

#         labels = countLabels([data1['data'], data2['data']])

        data1, label1 = getDataAndLabel(data1['data'])
        data2, label2 = getDataAndLabel(data2['data'])

        data1 = normalize(data1, axis = 0, norm = 'max').tolist()
        data2 = normalize(data2, axis = 0, norm = 'max').tolist()
        
        X_train += data1 + data2
        y_train += label1 + label2
        
        print ("subject " + str(subject_in) + (" finished. (train set)"))
        
    X_train = np.array(X_train, np.float32)
    y_train = np.array(y_train)
    X_test = np.array(X_test, np.float32)
    y_test = np.array(y_test)

subject 01 finished. (train set)
subject 02 finished. (train set)
subject 03 finished. (train set)
subject 04 finished. (train set)
subject 05 finished. (train set)
subject 06 finished. (train set)
subject 07 finished. (train set)
subject 08 finished. (train set)
subject 09 finished. (train set)
subject 10 finished. (test set)


## Siamese Neural Network
The following code is built upon the original code of https://github.com/NVIDIA/keras/blob/master/examples/mnist_siamese_graph.py

In [582]:
'''Train a Siamese MLP on pairs of digits from the MNIST dataset.
It follows Hadsell-et-al.'06 [1] by computing the Euclidean distance on the
output of the shared network and by optimizing the contrastive loss (see paper
for mode details).
[1] "Dimensionality Reduction by Learning an Invariant Mapping"
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
Gets to 99.5% test accuracy after 20 epochs.
3 seconds per epoch on a Titan X GPU
'''
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import RMSprop
from keras import optimizers
from keras.regularizers import l2
from keras.losses import binary_crossentropy
from keras import backend as K


def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 10
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


def create_pairs(x, crying_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(crying_indices[d]) for d in range(3)]) - 1
    
    for d in range(3):
        for i in range(n):
            z1, z2 = crying_indices[d][i], crying_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            
#             inc = random.randrange(1, 3)
            
            for inc in range(1, 3):
                dn = (d + inc) % 3

                z1, z2 = crying_indices[d][i], crying_indices[dn][i]
                pairs += [[x[z1], x[z2]]]
                
            z1, z2 = crying_indices[(d + 1) % 3][i], crying_indices[(d + 2) % 3][i]
            pairs += [[x[z1], x[z2]]]
            
            labels += [1, 0, 0, 0]
    return np.array(pairs), np.array(labels)


def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
#     seq.add(Dense(256, input_shape=(input_dim,), activation='relu'))
#     seq.add(Dropout(0.1))
#     seq.add(Dense(256, activation='relu'))
#     seq.add(Dropout(0.1))
#     seq.add(Dense(input_dim, activation='relu'))
    
    seq.add(Dense(512, input_shape=(input_dim,), activation='relu', kernel_regularizer=l2(2e-4)))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu', kernel_regularizer=l2(2e-4)))
    seq.add(Dropout(0.1))
    seq.add(Dense(512, activation='relu', kernel_regularizer=l2(2e-4)))
    seq.add(Dropout(0.1))
    seq.add(Dense(input_dim, activation='relu', kernel_regularizer=l2(1e-3)))
    return seq


def compute_accuracy(predictions, labels, threshold):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    
    return labels[predictions.ravel() < threshold].mean()



input_dim = 6373
epochs = 15

# create training+test positive and negative pairs
crying_indices = [np.where(y_train == i)[0] for i in range(3)]
tr_pairs, tr_y = create_pairs(X_train, crying_indices)

crying_indices = [np.where(y_test == i)[0] for i in range(3)]
te_pairs, te_y = create_pairs(X_test, crying_indices)

# network definition
base_network = create_base_network(input_dim)

input_a = Input(shape = (input_dim,))
input_b = Input(shape = (input_dim,))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape = eucl_dist_output_shape)([processed_a, processed_b])

model = Model(input = [input_a, input_b], output = distance)

# train
# rms = RMSprop()
adam = optimizers.Adam(lr=0.00005)
# adam = optimizers.Adam(lr=0.0001)
# model.compile(loss = 'binary_crossentropy', optimizer = adam)
model.compile(loss = contrastive_loss, optimizer = adam)
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          validation_data = ([te_pairs[:, 0], te_pairs[:, 1]], te_y),
          batch_size = 1024,
          epochs = epochs)

# # compute final accuracy on training and test sets
# pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
# tr_acc = compute_accuracy(pred, tr_y, 30.0)
# pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
# te_acc = compute_accuracy(pred, te_y, 30.0)

# print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
# print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

/Users/donghwashin/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:124: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("la..., inputs=[<tf.Tenso...)`


Train on 19212 samples, validate on 2124 samples
Epoch 1/15
19212/19212 [==============================] - 29s 1ms/step - loss: 62818.5082 - val_loss: 11107.9505
Epoch 2/15
19212/19212 [==============================] - 21s 1ms/step - loss: 15112.6763 - val_loss: 3329.1826
Epoch 3/15
19212/19212 [==============================] - 22s 1ms/step - loss: 6145.6633 - val_loss: 1488.0435
Epoch 4/15
19212/19212 [==============================] - 20s 1ms/step - loss: 3305.2750 - val_loss: 818.3093
Epoch 5/15
19212/19212 [==============================] - 20s 1ms/step - loss: 2077.8538 - val_loss: 489.1162
Epoch 6/15
19212/19212 [==============================] - 20s 1ms/step - loss: 1390.6860 - val_loss: 304.1999
Epoch 7/15
19212/19212 [==============================] - 20s 1ms/step - loss: 961.6946 - val_loss: 193.6558
Epoch 8/15
19212/19212 [==============================] - 20s 1ms/step - loss: 639.2710 - val_loss: 125.1480
Epoch 9/15
19212/19212 [==============================] - 20s 1ms/s

In [583]:
np.random.seed(124)

n_samples = 2
test_indices = []

crying_indices = [np.where(y_train == i)[0] for i in range(3)]
limit = min([len(crying_indices[d]) for d in range(3)]) - 1

# n_samples = min([len(crying_indices[d]) for d in range(3)]) - 1

for class_index in range(3):
#     test_indices.append(crying_indices[class_index][0:n_samples])
    test_indices.append(crying_indices[class_index][np.random.randint(limit, size = n_samples)])

In [584]:
result = []
predicted_class = []

lab_index = 0
total = len(X_test)

for record in X_test:
    min_class = 0
    min_dist = 987654321
    
    for index in range(3):
        test_data = X_train[test_indices[index]]
        predictions = model.predict([test_data, np.repeat([record], n_samples, axis=0)])
        
#         print (str(index) + ": " + str(predictions))
#         print (test_data)
        if (min_dist > predictions.mean()):
            min_dist = predictions.mean()
            min_class = index
            
    predicted_class.append(min_class)

    if min_class == y_test[lab_index]:
        result.append(1)
    else: result.append(0)
        
    lab_index += 1
    
    if lab_index % 150 == 0:
        print (str(lab_index / float(total) * 100) + "%")

print (np.array(result).mean())


5.28541226216%
10.5708245243%
15.8562367865%
21.1416490486%
26.4270613108%
31.7124735729%
36.9978858351%
42.2832980973%
47.5687103594%
52.8541226216%
58.1395348837%
63.4249471459%
68.710359408%
73.9957716702%
79.2811839323%
84.5665961945%
89.8520084567%
95.1374207188%
0.757928118393


In [585]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, predicted_class)

array([[2112,   23,  157],
       [ 311,    4,   53],
       [ 143,    0,   35]])